In [ ]:
path = r'C:\Users\Madao\Desktop\myData'
label_file = r'C:\Users\Madao\Desktop\labels.csv'

In [ ]:
import cv2
import os
import numpy as np

count = 0
images = []
classNo = []

my_list = os.listdir(path)
print('Total Classes Detected: ', len(my_list))

noOfClasses = len(my_list)
print('Importing Classes: ', noOfClasses)

for x in range(0, len(my_list)):
    myPicList = os.listdir(path + '\\' + str(count))
    for y in myPicList:
        curImg = cv2.imread(path + '\\' + str(count) + '\\' + y)
        images.append(curImg)
        classNo.append(count)
    print(count, end=' ')
    count += 1

print(' ')

images = np.array(images)
classNo = np.array(classNo)        

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, classNo, test_size=0.2)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
print(f'x_train {x_train.shape[0]} == y_train {y_train.shape}')
print(f'x_validation {x_validation.shape[0]} == y_validation {y_validation.shape}')
print(f'x_test {x_test.shape[0]} == y_test {y_test.shape}')
print(' ')
print(f'x_train {x_train.shape[-1]} == imageDimension {images[0].ndim}')
print(f'x_validation {x_validation.shape[-1]} == imageDimension {images[0].ndim}')
print(f'x_test {x_test.shape[-1]} == imageDimension {images[0].ndim}')

In [ ]:
import pandas as pd

df = pd.read_csv(label_file)
print('data shape ', df.shape, type(df))

In [ ]:
import matplotlib.pyplot as plt
import random

# num_of_samples = []
# cols = 5
# num_classes = noOfClasses

# fig, axs = plt.subplots(nrows=num_classes, ncols=cols, figsize=(5, 300))
# fig.tight_layout()

# for i in range(cols):
#     for j, row in df.iterrows():
#         x_selected = x_train[y_train == j]
#         axs[j][i].imshow(x_selected[random.randint(0, len(x_selected)- 1), :, :], cmap=plt.get_cmap('gray'))
#         axs[j][i].axis('off')
#         if i == 2:
#             axs[j][i].set_title(str(j) + '-' + row['Name'])
            # num_of_samples.append(len(x_selected))


In [ ]:
# for j,row in df.iterrows():
#     if j == 3:
#         break
#     print(f'j:{j} row:{row["Name"]}')

In [ ]:
# print(num_classes)
# plt.figure(figsize=(12, 4))
# plt.bar(range(0, num_classes), num_of_samples)
# plt.title('Distribution of the training dataset')
# plt.xlabel('Class number')
# plt.ylabel('Number of images')
# plt.show()

In [ ]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def preproccessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

In [ ]:
x_train = np.array(list(map(preproccessing, x_train)))
x_validation = np.array(list(map(preproccessing, x_validation)))
x_test = np.array(list(map(preproccessing, x_test)))

In [ ]:
# cv2.imshow('abc', x_train[9])

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_validation = x_validation.reshape(x_validation.shape[0], x_validation.shape[1], x_validation.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

dataGen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.2, rotation_range=10)
dataGen.fit(x_train)
batches = dataGen.flow(x_train, y_train, batch_size=20)
x_batch, y_batch = next(batches)

In [ ]:
fig, axs = plt.subplots(1, 15, figsize=(20, 5))
fig.tight_layout()

for i in range(15):
    axs[i].imshow(x_batch[i].reshape(32, 32))
plt.show()    

In [ ]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)

In [ ]:
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

def myModel():
    no_Of_Filters=60
    size_of_Filter=(5,5) # THIS IS THE KERNEL THAT MOVE AROUND THE IMAGE TO GET THE FEATURES.
                         # THIS WOULD REMOVE 2 PIXELS FROM EACH BORDER WHEN USING 32 32 IMAGE
    size_of_Filter2=(3,3)
    size_of_pool=(2,2)  # SCALE DOWN ALL FEATURE MAP TO GERNALIZE MORE, TO REDUCE OVERFITTING
    no_Of_Nodes = 500   # NO. OF NODES IN HIDDEN LAYERS
    model= Sequential()
    model.add((Conv2D(no_Of_Filters,size_of_Filter,input_shape=(imageDimesions[0],imageDimesions[1],1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE
    model.add((Conv2D(no_Of_Filters, size_of_Filter, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool)) # DOES NOT EFFECT THE DEPTH/NO OF FILTERS
 
    model.add((Conv2D(no_Of_Filters//2, size_of_Filter2,activation='relu')))
    model.add((Conv2D(no_Of_Filters // 2, size_of_Filter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(no_Of_Nodes,activation='relu'))
    model.add(Dropout(0.5)) # INPUTS NODES TO DROP WITH EACH UPDATE 1 ALL 0 NONE
    model.add(Dense(noOfClasses,activation='softmax')) # OUTPUT LAYER

    model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
batch_size_val=50
steps_per_epoch_val=2000
epochs_val=10
testRatio = 0.2
validationRatio = 0.2 
imageDimesions = (32,32,3)

In [ ]:
model = myModel()
print(model.summary())
history=model.fit(dataGen.flow(x_train,y_train,batch_size=batch_size_val),steps_per_epoch=steps_per_epoch_val,epochs=epochs_val,validation_data=(x_validation,y_validation),shuffle=1)

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('loss')
plt.xlabel('epoch')
plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Acurracy')
plt.xlabel('epoch')
plt.show()
score =model.evaluate(X_test,y_test,verbose=0)
print('Test Score:',score[0])
print('Test Accuracy:',score[1])

In [ ]:
import pickle

pickle_out= open("model_trained.p","wb")  # wb = WRITE BYTE
pickle.dump(model,pickle_out)
pickle_out.close()
cv2.waitKey(0)